#  Анализируем чеки

В этом задании мы будем работать с покупками и чеками. Смотреть за корреляциями в покупках довольно полезно.

> В 1992 году группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (Drug Store — формат разнокалиберных магазинов у дома). После анализа всех этих транзакций самым сильным правилом получилось «Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе». К сожалению, такое правило показалось руководству Osco Drug настолько контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали. Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с работы домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайший магазин. И мужья, не долго думая, совмещали приятное с полезным — покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения.

Для работы будем использовать датасет о продуктовых корзинах: https://www.kaggle.com/heeraldedhia/groceries-dataset

In [ ]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков

In [ ]:
!wget 'https://drive.google.com/u/0/uc?id=1SnaNzCXqNsNGtLTxLMw_MrureQN5NAy3' -O 'groceries.csv'

Подружаем данные и смотрим как они выглядят.

In [1]:
df = pd.read_csv('groceries.csv', sep=',')
df.columns = ['id', 'fielddate', 'product']
print(df.shape)
df.head()

(38765, 3)


     id   fielddate           product
0  1808  21-07-2015    tropical fruit
1  2552  05-01-2015        whole milk
2  2300  19-09-2015         pip fruit
3  1187  12-12-2015  other vegetables
4  3037  01-02-2015        whole milk

## 1. Корреляции

Для начала поработаем с корреляциями в данных. 

__а)__ Какой товар покупался чаще всего? Сохраните название этого товара в переменную `product_name`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

product_name = df['product'].value_counts().head(1).index[0]

In [ ]:
# проверка, что задание решено корректно
assert len(product_name) == 10

__б)__ Сколько всего уникальных заказов было сделано? Сохраните число заказов в переменную `n_cnt`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

n_cnt = df['id'].nunique()

In [ ]:
# проверка, что задание решено корректно
assert n_cnt > 3800
assert n_cnt < 4000

В таблице выше в каждой строке записана информация о покупке конкретного товара. Давайте сделаем табличку размера "число товаров" на "число покупок", чтобы понимать какие товары покупались вместе, а какие нет. 

> Обратите внимание, то здесь задание немного упрощено. Вообще говоря, нам нужно делать агрегацию по паре `fielddate, id`, если мы хотим изучать чеки по-честному. Но мы делаем её только по `id` для того, чтобы не усложнять задание. В качестве необязательного дополнения вы можете после сдачи задания переделать код так, чтобы дата тоже учитывалась при расчётах. 

In [2]:
sparse_sales = pd.pivot_table(df, 
               values='fielddate', 
               index='id', 
               columns='product', 
               fill_value=0, aggfunc='count')

sparse_sales.head()

product  Instant food products  UHT-milk  abrasive cleaner  ...  whole milk  yogurt  zwieback
id                                                          ...                              
1000                         0         0                 0  ...           2       1         0
1001                         0         0                 0  ...           2       0         0
1002                         0         0                 0  ...           1       0         0
1003                         0         0                 0  ...           0       0         0
1004                         0         0                 0  ...           3       0         0

[5 rows x 167 columns]

В нашей матрице огромное число нулей. Обычно такие матрицы называют разряжеными. Мы занимаем нулями кучу свободной памяти, которую мы могли бы не занимать, если бы хранили данные [в ином виде.](https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/)

__в)__ Постройте матрицу корреляций Пирсона. Для этого используйте метод таблицы `.corr`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

sales_correlation = sparse_sales.corr()

Какие продукты сильнее всего коррелируют с яйцами, `domestic eggs` (их чаще всего покупают вместе)?  Сохраните название самого скоррелированного продукта в переменную `top_1`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

top_1 = sales_correlation['domestic eggs'].nlargest(2).index[1]

Какие продукты "мешают" купить яйца, то есть отрицательно коррелируют с их покупкой? Сохраните название продукта с самой большой отрицательной корреляцией в переменную `bottom_1`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

bottom_1 = sales_correlation['domestic eggs'].nsmallest(1).index[0]

In [ ]:
# проверка, что задание решено корректно
assert len(bottom_1) == 8
assert len(top_1) == 12

Напишите код, который выводит самые коррелируемые товары для случайного продукта из списка `unique_products`.

In [3]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

unique_products = df['product'].unique()
random_products = sales_correlation['soups'].nlargest(3)[1:3].keys()
random_product = np.random.choice(random_products)

rp_corr = sales_correlation[random_product].nlargest(11)[1:]
rp_corr

product
soups               0.143465
margarine           0.039048
frankfurter         0.035177
bottled water       0.024923
soda                0.015311
other vegetables    0.011452
kitchen utensil    -0.000257
baby cosmetics     -0.000445
bags               -0.000513
make up remover    -0.000574
Name: preservation products, dtype: float64

__г)__ Какие два продукта коррелируют сильнее всего? Положите их название в лист `answer`

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = rp_corr.nlargest(2) 

In [ ]:
# проверка, что задание решено корректно
assert 'soups' in answer

Конечно же, корреляция — это [не единственный способ искать](https://habr.com/ru/company/ods/blog/353502/) между покупками ассоциативные правила.

## 2. Зависимость. 

В лекции мы с вами сказали, что события $A$ и $B$ называются независимыми, если $P(AB) = P(A)\cdot P(B)$. Отталкиваясь от этого определения, можно ввести другую характеристику, которая показывает, насколько продукты зависят друг от друга, а именно __поддержку (lift).__ 

$$
lift = \frac{P(AB)}{P(A)\cdot P(B)}
$$

Эта метрика описывает отношение зависимости товаров к их независимости. Если оказалось, что `lift = 1`, это означает, что покупка товара $A$ не зависит от покупки товара $B$. Если `lift > 1`, то это означает, что вероятность встретить оба товара в чеке, $P(AB)$ высокая, то есть товары покупают вместе. Если `lift < 1`, это означает, что товары, наоборот, очень часто покупают по-отдельности. 

__д)__ Посчитайте значение нашей метрики для яиц и молока (`'whole milk', 'domestic eggs'`). Запишите получившиеся значение метрики в переменную `answer`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

sales_pa = (sparse_sales['whole milk'] != 0).mean()
sales_pb = (sparse_sales['domestic eggs'] != 0).mean()
sales_pab = ((sparse_sales['whole milk'] != 0) & (sparse_sales['domestic eggs'] != 0)).mean()

answer = sales_pab / (sales_pa * sales_pb)

In [ ]:
# проверка, что задание решено корректно
assert answer < 3
assert answer > 1

__е)__ Посчитайте значение метрики для всех пар продуктов из датасета. Сохраните значения в словарик `dict`. В качестве ключа используете кортеж из пары продуктов. Чтобы удобнее было перебрать все сочетания, используйте `combinations` из модуля `itertools`.

Чтобы при подсчётах не возникало деления на ноль, добавьте к знаменателю маленькое число, например `1e-10`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

def lift(a, b):
    pa = (sparse_sales[a] != 0).mean()
    pb = (sparse_sales[b] != 0).mean()
    pab = ((sparse_sales[a] != 0) & (sparse_sales[b] != 0)).mean()

    return pab / (pa * pb)

unvisited_products = list(unique_products)
food_lift = {}

while len(unvisited_products):
    product = unvisited_products[0]
    for i in range(1, len(unvisited_products)):
        subproduct = unvisited_products[i]
        food_lift[(product, subproduct)] = lift(product, subproduct)
    unvisited_products.pop(0)

In [ ]:
import itertools as it
comb = list(it.combinations(unique_products, 2))

In [4]:
test = []
for i in range(len(comb)):
    t = len(sparse_sales[(sparse_sales[comb[i][0]]==True)&(sparse_sales[comb[i][1]]==True)])
    if (t == 0) and food_lift[comb[i]] > 0:
        print(comb[i])
        break
    if t != 0:
        test.append(comb[i])

('whole milk', 'cream')


In [5]:
comb[i]

('whole milk', 'cream')

In [6]:
sparse_sales[(sparse_sales[comb[i][0]]==True)&(sparse_sales[comb[i][1]]==True)]

Empty DataFrame
Columns: [Instant food products, UHT-milk, abrasive cleaner, artif. sweetener, baby cosmetics, bags, baking powder, bathroom cleaner, beef, berries, beverages, bottled beer, bottled water, brandy, brown bread, butter, butter milk, cake bar, candles, candy, canned beer, canned fish, canned fruit, canned vegetables, cat food, cereals, chewing gum, chicken, chocolate, chocolate marshmallow, citrus fruit, cleaner, cling film/bags, cocoa drinks, coffee, condensed milk, cooking chocolate, cookware, cream, cream cheese , curd, curd cheese, decalcifier, dental care, dessert, detergent, dish cleaner, dishes, dog food, domestic eggs, female sanitary products, finished products, fish, flour, flower (seeds), flower soil/fertilizer, frankfurter, frozen chicken, frozen dessert, frozen fish, frozen fruits, frozen meals, frozen potato products, frozen vegetables, fruit/vegetable juice, grapes, hair spray, ham, hamburger meat, hard cheese, herbs, honey, house keeping products, hygiene a

In [7]:
food_lift[('whole milk', 'cream')]

0.5456326987681971

In [8]:
(sparse_sales [['whole milk', 'cream']] >= 1).all(axis=1).sum()

3

In [9]:
ab = (sparse_sales [['whole milk', 'cream']] >= 1).all(axis=1).mean()
a = (sparse_sales ['whole milk'] >= 1).mean()
b = (sparse_sales ['cream'] >= 1).mean()
answer = ab/(a*b)
answer

0.5456326987681971

In [10]:
len(set(test))

303

Сколько пар продуктов покупали вместе хотя бы раз? Запишите ответ в переменную `answer`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = 0
for _, value in food_lift.items():
    if value: answer += 1

Для какой пары продуктов метрика $lift$ оказалась самой большой? 

In [11]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

food_lift_largest = max(food_lift, key=food_lift.get)
food_lift_largest

# your code here

('soups', 'preservation products')

Сколько раз эти продукты встретились в выборке? Как думаете адеватно ли делать выводы по такому объёму данных? 

In [12]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

print(food_lift_largest[0], ':', sparse_sales[food_lift_largest[0]].sum())
print(food_lift_largest[1], ':', sparse_sales[food_lift_largest[1]].sum())

soups : 48
preservation products : 1


Для какой пары продуктов метрика оказывается самой маленькой? 

In [13]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

food_lift_smallest = min(food_lift, key=food_lift.get)
food_lift_smallest

# your code here

('tropical fruit', 'hair spray')

In [ ]:
# проверка, что задание решено корректно

assert answer < 10000
assert answer > 9000

## 3. Неоцениваемые задания

Выше мы увидели, что некоторые продукты встречаются в выборке очень редко. Понятное дело, что по ним у нас не получится построить хорошее ассоциативное правило. Попробуйте повторить расчёт той же метрики, но с условием что продукт покупали больше 10 раз. Изучите самые покупаемые вместе продукты и самые непокупаемые вместе продукты. Насколько сильно список отличается от полученного в предыдущем задании? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

product_cnt = df['product'].value_counts()
unique_products = product_cnt[product_cnt > 10].keys()
sparse_sales = sparse_sales[unique_products]
comb = list(it.combinations(unique_products, 2))

In [ ]:
unvisited_products = list(unique_products)
food_lift = {}

for i in range(len(comb)):
    food_lift[(comb[i][0], comb[i][1])] = lift(comb[i][0], comb[i][1])

In [14]:
import heapq

top10 = heapq.nlargest(10, food_lift, key=food_lift.get)
bottom10 = heapq.nsmallest(10, food_lift, key=food_lift.get)

print("Top 10:")
for i in top10:
    print(i, food_lift[i])

print("Bottom 10:")
for i in bottom10:
    print(i, food_lift[i])

Top 10:
('ready soups', 'honey') 19.98974358974359
('tea', 'nut snack') 19.68686868686869
('ketchup', 'honey') 18.740384615384617
('cooking chocolate', 'ready soups') 17.324444444444445
('potato products', 'canned fruit') 16.874458874458874
('tidbits', 'frozen fruits') 16.107438016528928
('prosecco', 'honey') 15.781376518218625
('skin care', 'honey') 14.99230769230769
('prosecco', 'cooking chocolate') 13.677192982456141
('flower (seeds)', 'honey') 13.425947187141217
Bottom 10:
('bottled water', 'cream') 0.0
('pip fruit', 'ready soups') 0.0
('canned beer', 'frozen fruits') 0.0
('bottled beer', 'flower soil/fertilizer') 0.0
('bottled beer', 'cream') 0.0
('frankfurter', 'frozen fruits') 0.0
('brown bread', 'ready soups') 0.0
('pork', 'kitchen towels') 0.0
('pork', 'organic sausage') 0.0
('pork', 'frozen fruits') 0.0


Иногда в чеках пытаются искать __продукты-якоря.__ То есть продукты, которые являются основными. Например: айфон - основной продукт, наушники и чехол - дополнения к нему. Подумайте как можно попытаться найти такие продукты на основе простых метрик, основанных на подсчёте условных вероятностей.

<center>
<img src="https://pp.userapi.com/c638028/v638028181/52e5e/1X-dkzNN1hk.jpg" width="400"> 
</center>